# MAD Dynamique <br>
### STAGE ANCHES <br>
Camille Urban <br>
22/03/2024

In [2]:
# classic shit
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
from scipy.signal import butter, filtfilt

#librosa
import librosa
import librosa.display

# traitement image
from skimage.io import imread
from skimage.transform import rotate
# from im_rotate import im_rotate
import os

In [3]:
%matplotlib
%matplotlib

Using matplotlib backend: <object object at 0x00000220B97BE360>
Using matplotlib backend: TkAgg


## Paramètres du Banc

In [4]:
# SENSIBILITE CAPTEURS
S_mic_int =  0.0841  # Sensibilité du capteur de pression interne (mV/Pa)
Offset_mic_int = 51.2   # (mV) 
# G_micro_int = 10 V
S_force = 39.2 # mV/N
# Offset_force = 
# G_force = 10 V
# étalonnage de micro exterieur par source étalon 1kHz à 97dB

# FRQ ECHANTILLONNAGE
sr = 10000 
n_fft = 512 # env. 50ms
hop_length = n_fft / 4  # = 12.5ms (pas de décalage de la fenêtre)
p_ref = 2e-5


In [5]:
dossier = 'mes_2024.04.19/'
nom_fichier = 'anche'
nombre_anches = 5

# Traitement données

### Extraction de données et plots temporels

In [6]:
# normalisation 
def normalisation(signal):
    # Normaliser le signal de pression par sa pression moyenne
    # normalized_signal = signal / np.mean(signal)
    
    # Normaliser le signal de pression par sa pression maximale
    normalized_signal = signal / np.max(signal)
    
    return normalized_signal

In [7]:
# for i in range (3):
#     A = pd.read_table(f'MAD Dynamique\mesures 2024.03.20\mes_2024.03.20\Anche{i+1}', sep='\t', header=2)
plt.close('all')
A = []
temps = []
forces = []
p_ints = []
p_exts = []

for i in range (nombre_anches):
    plt.figure()
    data = pd.read_table( dossier + nom_fichier + f'{i+1}', sep='\t', header=2)
    # A.append(data)
    A = np.array(data)
    # print(A)
    tps = A[:,0]
    tps = normalisation(tps)
    temps.append(tps)
    force = A[:,1]/S_force
    force = normalisation(force)
    forces.append(force)
    p_int = (A[:,2])/S_mic_int
    p_int = normalisation(p_int)
    p_ints.append(p_int)
    p_ext = A[:,3]
    p_ext = normalisation(p_ext)
    p_exts.append(p_ext)

    # plt.plot(tps, A[:,1], label='force')
    plt.plot(temps[i], p_ints[i], label='micro bec')
    plt.plot(temps[i], p_exts[i], label='micro ext')
    
plt.xlabel('temps (s)')
plt.ylabel('Amplitude (Pa)')
plt.legend()
plt.show()

### Spectrogramme des signaux en pression (microphone interne et externe)

In [8]:

# Supposons que vous avez une matrice d'échantillons audio appelée 'y' et un taux d'échantillonnage appelé 'sr'
# y est une matrice représentant les échantillons audio
# sr est le taux d'échantillonnage (nombre d'échantillons par seconde)
plt.close('all')

p_int = p_ints[4]
p_ext = p_exts[4]

# Calculer le spectrogramme
spectrogram_int = np.abs(librosa.stft(y=p_int, n_fft=n_fft))
spectrogram_ext = np.abs(librosa.stft(y=p_ext, n_fft=n_fft))


# Convertir l'échelle en décibels (dB)
spectrogram_int_db = librosa.power_to_db(spectrogram_int**2, ref=p_ref**2)
spectrogram_ext_db = librosa.power_to_db(spectrogram_ext**2, ref=p_ref**2)

# Afficher le spectrogramme
fig, (pres_int, pres_ext) = plt.subplots(1, 2, figsize=(10, 4))

img_int = librosa.display.specshow(spectrogram_int_db, hop_length=hop_length, x_axis='time', y_axis='mel', ax=pres_int)
# p_int.set_colorbar(format='%+2.0f dB')
cbar = fig.colorbar(img_int, ax=pres_int, format='%+2.0f dB')
cbar.set_label('Amplitude (dB)')
pres_int.set_title('Pression interne au bec')

img_ext = librosa.display.specshow(spectrogram_ext_db, hop_length=hop_length, x_axis='time', y_axis='mel', ax=pres_ext)
cbar = fig.colorbar(img_ext, ax=pres_ext, format='%+2.0f dB')
cbar.set_label('Amplitude (dB)')
pres_ext.set_title('Pression externe au bec')

fig.suptitle('Spectrogrammes', fontsize=20)
plt.tight_layout()
plt.show()


### Enveloppe des signaux en pression et detection de seuil

#### Fonctions : <br>
- trouver la fréquence fondamentale 
- filtrer le signale fréquentiellement autour de la fondamentale
- calcul de l'enveloppe spectrale

In [9]:
# FILTRER LE SIGNAL AUTOUR DE LA FREQUENCE FONDAMENTALE

# Retourne la valeur de la freq fondamentale
def freq_fond(signal) : 
    # TF
    TF = np.fft.fft(signal)

    # Calcul des fréquences correspondantes
    freqs = np.fft.fftfreq(len(signal), d=1/sr)

    # Indice de la fréquence fondamentale (la fréquence avec l'amplitude maximale, à l'exception de la première fréquence nulle)
    indice_fonda = np.argmax(np.abs(TF[1:])) + 1

    # La fréquence fondamentale correspondante est la fréquence à cet indice
    freq_fonda = np.abs(freqs[indice_fonda])
    print("Fréquence fondamentale détectée:", freq_fonda)
    return freq_fonda

    
# Retourne le signal filtré autour de la fréquence fondamentale
# Prend en entrée le signal et la largeur de la bande passante autour de la fréquence fondamentale
def filtered_sig(signal,largeur_bande) :
    # Appeler la fct qui cherhce la frequence fondamenetale
    freq_fonda = freq_fond(signal)
    # print(freq_fonda)
    
    # Definition de l'ordre du filtre
    ordre_filtre = 4  
    
    # Calcul des fréquences de coupure
    freq_coup_inf = freq_fonda - largeur_bande / 2
    freq_coup_sup = freq_fonda + largeur_bande / 2

    # fréquences de coupure en fréquences normalisées
    frequence_coupure_inf_norm = freq_coup_inf / (sr / 2)
    frequence_coupure_sup_norm = freq_coup_sup / (sr / 2)

    # Création du filtre passe-bande
    b, a = butter(ordre_filtre, [frequence_coupure_inf_norm, frequence_coupure_sup_norm], btype='band')

    # Filtrage du signal
    signal_filtre = filtfilt(b, a, signal)
    return signal_filtre


In [10]:
# CALCUL DE L'ENVELOPPE SPECTRALE

# Fonction pour calculer l'enveloppe spectrale par convolution avec fenêtre de Hann
def spectral_envelope(signal, window_size):
    # Calculer le nombre de segments
    num_segments = len(signal) // window_size
    
    # Initialiser l'enveloppe spectrale
    spectral_env = np.zeros_like(signal)
    
    # fenêtrage de Hann sur chaque segment et TF
    for i in range(num_segments):
        start_idx = i * window_size
        end_idx = (i + 1) * window_size
        segment = signal[start_idx:end_idx]
        
        # fenêtrage
        segment *= sp.signal.windows.hann(window_size)
        
        # TF et amplitude
        spectrum = np.fft.fft(segment)
        amplitude = np.abs(spectrum)
        
        spectral_env[start_idx:end_idx] = amplitude
    
    # Lissage par convolution
    spectral_env = np.convolve(spectral_env, np.ones(window_size) / window_size, mode='same')
    
    return spectral_env

In [11]:
# VARIABLES 

n=1
signal = p_ints[n]
tps = temps[n]

# APPEL FONCTION FILTRAGE
freq_bande = 100  

# Seuil de détection absolu
seuil = 0.01
 
# Fréquence de coupure (Hz)
F_c = 20

# Taille de la fenêtre de Hann
# window_size = n_fft
window_size = int(sr / F_c)

In [12]:
plt.close('all')

# APPEL FONCTION FILTRAGE
signal_filtre = filtered_sig(signal, freq_bande)

################################################################
# tracer le signal initial et le signal filté
plt.figure()
plt.plot(tps, signal, label='Signal initial')
plt.plot(tps, signal_filtre, label='Signal filtré')
plt.xlabel('Temps')
plt.ylabel('Amplitude')
plt.title('Comparaison entre le signal initial et le signal filtré')
plt.legend()
plt.show()

################################################################
plt.close('all')

# Calcul enveloppe spectrale
# spectral_env = spectral_envelope(signal, window_size)
spectral_env = spectral_envelope(signal_filtre, window_size)

# Normalisation de l'enveloppe
spectral_env_normalized = spectral_env / np.max(spectral_env)

# Afficher le signal et son enveloppe spectrale
plt.figure(figsize=(10, 6))
# plt.plot(tps, signal, label='Signal normalisé')           # signal brut
# plt.plot(tps, np.abs(signal), label='Signal normalisé')   # val absolue du signal
plt.plot(tps, signal_filtre, label='Signal normalisé')      # signal filtre autour de la ère freq fonda
plt.plot(tps, spectral_env_normalized, label='Enveloppe spectrale')
plt.hlines(seuil, tps[0], tps[-1], label=f'seuil {seuil}')
plt.xlabel('Temps')
plt.ylabel('Amplitude')
plt.title('Enveloppe spectrale par convolution avec fenêtre de Hann')
plt.legend()
plt.grid(True)
plt.show()


#################################################################

# TRONCATURE DU SIGNAL

# Initialiser une liste pour stocker les indices de début et de fin des segments détectés
segments = []

seuil_depasse = False
for i, valeur in enumerate(spectral_env_normalized):
    if not seuil_depasse and valeur > seuil:
        seuil_depasse = True
        segments.append(i)  # Ajouter le moment où le seuil est dépassé
    elif seuil_depasse and valeur <= seuil:
        seuil_depasse = False
        segments.append(i)   

print(segments)


# Marge autour des indices de détection de seuil 
marge = 200  

# Tronquer le signal autour des indices de détection de seuil
signal_tronc_mont = signal[segments[0] - marge:segments[1] + marge]
temps_tronc_mont = tps[segments[0] - marge:segments[1] + marge]
# Tronquer le signal autour des indices de détection de seuil
signal_tronc_desc = signal[segments[2] - marge:segments[-1] + marge+400]
temps_tronc_desc = tps[segments[2] - marge : segments[-1] + marge+400]

# Calculer la somme cumulée des énergies normalisées
e_cum = np.cumsum(spectral_env_normalized)

# passage de seuil
passage_seuil = []

# Parcourir la somme cumulée des énergies
for i in range(1, len(e_cum)):
    # Si la somme cumulée dépasse le seuil absolu
    if e_cum[i] >= seuil and e_cum[i - 1] < seuil:
        passage_seuil.append(i)  # Ajouter l'indice où la somme d'énergie cumulée passe au-dessus du seuil
    # Si la somme cumulée passe en dessous du seuil absolu
    elif e_cum[i] < seuil and e_cum[i - 1] >= seuil:
        passage_seuil.append(i)

e_cum = normalisation(e_cum)

plt.figure(figsize=(10, 6))
plt.plot(tps, signal_filtre, label='Signal normalisé')
plt.plot(tps, spectral_env_normalized, label='Enveloppe spectrale')
plt.plot(tps, e_cum, label='somme cumulée des énergie')
plt.grid(True)
plt.xlabel('Temps (s)')
plt.ylabel('Energie')
plt.title('Energie cumulative')
plt.legend()

plt.figure(figsize=(10, 6))
plt.plot(temps_tronc_mont, signal_tronc_mont, label='Signal normalisé')
plt.grid(True)
plt.xlabel('Temps (s)')
plt.ylabel('Amplitude normalisée')
plt.title('Signal tronqué - pression croissante')
plt.legend()

plt.figure(figsize=(10, 6))
plt.plot(temps_tronc_desc, signal_tronc_desc, label='Signal normalisé')
plt.grid(True)
plt.xlabel('Temps (s)')
plt.ylabel('Amplitude normalisée')
plt.title('Signal tronqué - pression croissante')
plt.legend()


Fréquence fondamentale détectée: 209.6620570753409
[19262, 36241, 45261, 63761]


In [116]:
plt.close('all')


def signal_statio(temps, signal) : 
    E_t = sp.integrate.cumtrapz(y=signal**2, x=temps, dx = 1/sr)
    
    indices_statio = [np.argmax(E_t >= 0.05 * np.max(E_t)), np.argmax(E_t > 0.95 * np.max(E_t))-1 ]
    tps_statio = temps[indices_statio]
    E_t_statio = E_t[indices_statio]
    
    print(indices_statio)
    print(tps_statio)
    print(E_t_statio)
    return E_t, indices_statio, tps_statio, E_t_statio


signals_troncs = [signal_tronc_mont, signal_tronc_desc]
temps_troncs = [temps_tronc_mont, temps_tronc_desc]
labels = ['Début du signal staionnaire \n$0.05*E(t)$', 'Fin du signal staionnaire \n$0.95*E(t)$']
color = ['orange', 'green']

for i in range(2):
    x = temps_troncs[i]
    y = signals_troncs[i]
    E_t, indices_statio, tps_statio, E_t_statio = signal_statio(x, y)
    plt.figure(figsize=(10, 6))
    plt.plot(temps_troncs[i][0:-1], E_t, label='Signal normalisé') 
    for j in range (2): 
        x = tps_statio[j]
        y = E_t_statio[j]
        plt.vlines(x, 1.2*np.min(E_t), 1.2*np.max(E_t), colors=color[j], label=labels[j])
        plt.hlines(y, temps_troncs[i][0], temps_troncs[i][-1], colors=color[j])
        plt.scatter(x, y, color=color[i])
        plt.text(x, y, f'({round(x,2)}, {np.format_float_scientific(round(y,4))})', fontsize=10)
        plt.grid(True)
        plt.xlabel('Temps (s)')
        plt.ylabel('Amplitude normalisée')
        plt.title('Energie en fonction du temps $E(t) = \int_{t_{deb}}^{t_{fin}} p^{2}(t) d\\tau$')
        plt.legend()
    

################################################################ on peut supprimer 
# MONTEE
x = temps_tronc_mont
y = signal_tronc_mont
E_t, indices_statio, tps_statio, E_t_statio = signal_statio(x, y)


# FIGURE
plt.figure(figsize=(10, 6))
plt.plot(temps_tronc_mont[0:-1], E_t, label='Signal normalisé')   
    
# début de la partie stationnaire
x= tps_statio[0]
y = E_t_statio[0]
plt.vlines(x, 1.2*np.min(E_t), 1.2*np.max(E_t), colors=color[i], label='Début du signal staionnaire \n$0.05*E(t)$')
plt.hlines(y, temps_tronc_mont[0], temps_tronc_mont[-1], colors='orange')
plt.scatter(x, y, color='orange')
plt.text(x, y, f'({round(x,2)}, {np.format_float_scientific(round(y,4))})', fontsize=10)

# fin de la partie stationnaire
x= tps_statio[1]
y = E_t_statio[1]
plt.vlines(x, 1.2*np.min(E_t), 1.2*np.max(E_t), colors='purple', label='Fin du signal staionnaire \n$0.95*E(t)$')
plt.hlines(y, temps_tronc_mont[0], temps_tronc_mont[-1], colors='purple')
plt.scatter(x, y, color='purple')
plt.text(x, y, f'({round(x,2)}, {np.format_float_scientific(round(y,4))})', fontsize=10)

plt.grid(True)
plt.xlabel('Temps (s)')
plt.ylabel('Amplitude normalisée')
plt.title('Energie en fonction du temps $E(t) = \int_{t_{deb}}^{t_{fin}} p^{2}(t) d\\tau$')
plt.legend()

################################################################
# MONTEE
x = temps_tronc_desc
y = signal_tronc_desc
E_t, indices_statio, tps_statio, E_t_statio = signal_statio(x, y)
print('début du signal stationnaire $t_{deb}$' )

# FIGURE
plt.figure(figsize=(10, 6))
plt.plot(temps_tronc_desc[0:-1], E_t, label='Signal normalisé')

# début de la partie stationnaire
x= tps_statio[0]
y = E_t_statio[0]
plt.vlines(x,  1.2*np.min(E_t), 1.2*np.max(E_t), colors='orange', label='Début du signal staionnaire \n$0.05*E(t)$')
plt.hlines(y, temps_tronc_desc[0], temps_tronc_desc[-1], colors='orange')
plt.scatter(x, y, color='orange')
plt.text(x, y, f'({round(x,2)}, {np.format_float_scientific(round(y,4))})', fontsize=10)

# fin de la partie stationnaire
x= tps_statio[1]
y = E_t_statio[1]
plt.vlines(x, 1.2*np.min(E_t), 1.2*np.max(E_t), colors='purple', label='Fin du signal staionnaire \n$0.95*E(t)$')
plt.hlines(y, temps_tronc_desc[0], temps_tronc_desc[-1], colors='purple')
plt.scatter(x, y, color='purple')
plt.text(x, y, f'({round(x,2)}, {np.format_float_scientific(round(y,4))})', fontsize=10)

plt.grid(True)
plt.xlabel('Temps (s)')
plt.ylabel('Amplitude normalisée')
plt.title('Energie en fonction du temps $E(t) = \int_{t_{deb}}^{t_{fin}} p^{2}(t) d\\tau$')
plt.legend()

C:\Users\Admin\AppData\Local\Temp\ipykernel_15540\1513099923.py:5: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  E_t = sp.integrate.cumtrapz(y=signal**2, x=temps, dx = 1/sr)


[3065, 13815]
[0.26678562 0.3963927 ]
[0.00342068 0.06495272]
[1716, 13085]
[0.56397767 0.70104771]
[0.00176212 0.03342077]
[3065, 13815]
[0.26678562 0.3963927 ]
[0.00342068 0.06495272]
[1716, 13085]
[0.56397767 0.70104771]
[0.00176212 0.03342077]
début du signal stationnaire $t_{deb}$


In [21]:
plt.close('all')

# y = p_ext
# S, phase = librosa.magphase(librosa.stft(y)) # S=p_int

rms = librosa.feature.rms(S=spectrogram_int, frame_length = n_fft).squeeze() # A mettre en dB 
print(rms.shape)
rms_diff = np.diff(rms, 1)
max_diff = np.abs(np.max(rms_diff))
ind = np.where(rms_diff>0.5*max_diff)[0]
print(ind*hop_length/sr)
seuil_0_diff = 0.00005
ind2 = np.where(np.abs(rms_diff)<seuil_0_diff)[0]
print(ind2*hop_length/sr)

plt.figure()

times = librosa.times_like(rms)
plt.plot(times, rms, label='RMS Energy')
times = librosa.times_like(rms_diff)
plt.plot(times, rms_diff, label='RMS Energy')
plt.legend()

# librosa.display.specshow(spectrogram_int_db, y_axis='linear', x_axis='time', ax=ax[1])

# ax[2].plot(tps, A[:,3], label='micro bec')

(704,)
[1.2032 1.216  1.2288 6.016  6.0288 6.0416 6.0544 6.0672 6.08  ]
[0.3712 0.4096 1.1392 1.8048 1.9968 2.0864 2.1632 2.304  2.5344 2.5856
 2.6624 2.7264 2.9056 5.4528 5.6192 5.632  5.6576 8.0768 8.1024 8.1664
 8.4992 8.6272 8.6528 8.832  8.8448]


In [20]:
plt.close('all')

# y = p_ext
# S, phase = librosa.magphase(librosa.stft(y)) # S=p_int

rms = librosa.feature.rms(S=spectrogram_int, frame_length = n_fft).squeeze() # A mettre en dB 
print(rms.shape)
rms_diff = np.diff(rms, 1)
max_diff = np.abs(np.max(rms_diff))
ind = np.where(rms_diff>0.5*max_diff)[0]
print(ind*hop_length/sr)
seuil_0_diff = 0.00005
ind2 = np.where(np.abs(rms_diff)<seuil_0_diff)[0]
print(ind2*hop_length/sr)

fig, ax = plt.subplots(nrows=2, sharex=True)

times = librosa.times_like(rms)
ax[0].plot(times, rms, label='RMS Energy')
times = librosa.times_like(rms_diff)
ax[0].plot(times, rms_diff, label='RMS Energy')
ax[0].set(xticks=[])
ax[0].legend()
ax[0].label_outer()

librosa.display.specshow(spectrogram_int_db, y_axis='linear', x_axis='time', ax=ax[1])
ax[1].set(title='log Power spectrogram')

# ax[2].plot(tps, A[:,3], label='micro bec')

(648,)
[2.0224 4.608  4.6208 4.6336 4.6464]
[2.944 8.   ]


[Text(0.5, 1.0, 'log Power spectrogram')]

In [41]:

plt.close('all')
fig, axs = plt.subplots(3, 2, figsize=(12, 7))

# FFT
fft_int = np.fft.fft(p_int)
fft_freq_int = np.fft.fftfreq(p_int.shape[-1])
axs[0, 0].plot(fft_freq_int, fft_int)
axs[0, 0].set_xlabel('Frequence (Hz)')
axs[0, 0].set_ylabel('Amplitude (mV)')
axs[0, 0].set_title('Intérieur Bec \n FFT')
axs[0, 0].set_xlim([0, 0.4])
axs[0, 0].set_ylim([0, None])

fft_ext = np.fft.fft(p_ext)
fft_freq_ext = np.fft.fftfreq(p_ext.shape[-1])
axs[0, 1].plot(fft_freq_ext, fft_ext)
axs[0, 1].set_xlabel('Frequence (Hz)')
axs[0, 1].set_ylabel('Amplitude (mV)')
axs[0, 1].set_title('Extérieur Bec \n FFT')
axs[0, 1].set_xlim([0, 0.4])
axs[0, 1].set_ylim([0, None])

# SIGNAUX TEMPORELS
axs[1, 0].plot(tps, p_int, label='micro ext')
S, phase = librosa.magphase(librosa.stft(p_int))
rms = librosa.feature.rms(S=S)
times = librosa.times_like(rms)
# axs[1, 0].plot(times, rms[0], label='RMS Energy')
axs[1, 0].set_title('Signal temporel')
axs[1, 0].set_xlabel('Temps (s)')
axs[1, 0].set_ylabel('Amplitude (mV)')

axs[1, 1].plot(tps, p_ext, label='micro int')
axs[1, 1].set_title('Signal temporel')
axs[1, 1].set_xlabel('Temps (s)')
axs[1, 1].set_ylabel('Amplitude (mV)')


# SPRECTRO
spectrogram_int = librosa.feature.melspectrogram(y=p_int)
spectrogram_ext = librosa.feature.melspectrogram(y=p_ext)

# Convertir l'échelle en décibels (dB)
spectrogram_int_db = librosa.power_to_db(spectrogram_int, ref=np.max)
spectrogram_ext_db = librosa.power_to_db(spectrogram_ext, ref=np.max)

img_int = librosa.display.specshow(spectrogram_int_db, ax=axs[2, 0])
# p_int.set_colorbar(format='%+2.0f dB')
# cbar = fig.colorbar(img_int, ax=axs[2,0], format='%+2.0f dB')
# cbar.set_label('Amplitude (dB)')
axs[2, 0].set_title('Pression interne au bec')
axs[2, 0].set_xlabel('Temps (s)')
axs[2, 0].set_ylabel('Frequence (Hz)')

img_ext = librosa.display.specshow(spectrogram_ext_db, x_axis='time', y_axis='mel', ax=axs[2, 1])
# cbar = fig.colorbar(img_ext, ax=axs[2, 1], format='%+2.0f dB')
cbar.set_label('Amplitude (dB)')
axs[2, 1].set_title('Pression externe au bec')
axs[2, 1].set_xlabel('Temps (s)')
axs[2, 1].set_ylabel('Frequence (Hz)')

# CGS_int = librosa.feature.spectral_centroid(y=p_int)
# CGS_ext = librosa.feature.spectral_centroid(y=p_ext)
# axs[3,0].plot(CGS_int.squeeze())
# axs[3,1].plot(CGS_ext.squeeze())

# fig.suptitle('Spectrogrammes', fontsize=20)
plt.tight_layout()
plt.show()

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\matplotlib\cbook.py:1699: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\matplotlib\cbook.py:1345: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [73]:
plt.close('all')
plt.figure()

# on peut aussi demander à librosa de traduire notre temps (tps) en frame directement
# times = librosa.times_like(CGS_int)
print(len(tps))
# t = librosa.time_to_frames(tps, sr=sr, hop_length=n_fft//4)
# print(t.shape)
t_max = len(p_int)//sr
N = len(p_int)//n_fft
print(N)
t = np.linspace(0, t_max, N)



CGS_int = librosa.feature.spectral_centroid(y=p_int, sr=sr, n_fft=n_fft).squeeze()
CGS_ext = librosa.feature.spectral_centroid(y=p_ext, sr=sr, n_fft=n_fft).squeeze()
n=min(len(t), len(CGS_int))
print(n)

print(CGS_int.shape)
plt.plot(t[:n], CGS_int[:n], label='int')
plt.plot(t[:n], CGS_ext[:n], label='ext')
plt.xlabel('Time')
plt.ylabel('CGS (Hz)')
plt.legend()
# reg = LinearRegression()
# reg.fit(force,CGS)
# reg.score(X,CGS)

68863
134
134
(135,)


In [ ]:

# Charger le fichier audio
signal = p_ext
# sr = None

# Extraire les descripteurs audio
energie = librosa.feature.rms(y=signal)
mfcc = librosa.feature.mfcc(y=signal, n_mfcc=13)
chroma = librosa.feature.chroma_stft(y=signal)
zero_crossing_rate = librosa.feature.zero_crossing_rate(y=signal)
spectral_centroid = librosa.feature.spectral_centroid(y=signal)
spectral_contrast = librosa.feature.spectral_contrast(y=signal)
spectral_rolloff = librosa.feature.spectral_rolloff(y=signal)
mfcc_delta = librosa.feature.delta(mfcc)
mfcc_delta_delta = librosa.feature.delta(mfcc, order=2)
centres_bandes = librosa.feature.spectral_centroid(y=signal, n_fft=2048, hop_length=512)

# Afficher les dimensions de chaque descripteur
# print("Énergie du signal:", energie.shape)
print("MFCC:", mfcc.shape)
# print("Chroma:", chroma.shape)
print("Zero-Crossing Rate:", zero_crossing_rate.shape)
print("Spectral Centroid:", spectral_centroid.shape)
print("Spectral Contrast:", spectral_contrast.shape)
print("Spectral Rolloff:", spectral_rolloff.shape)
print("MFCC Delta:", mfcc_delta.shape)
print("MFCC Delta-Delta:", mfcc_delta_delta.shape)
print("Centres de bande de fréquences:", centres_bandes.shape)



Énergie du signal: (1, 194)
MFCC: (13, 194)
Chroma: (12, 194)
Zero-Crossing Rate: (1, 194)
Spectral Centroid: (1, 194)
Spectral Contrast: (7, 194)
Spectral Rolloff: (1, 194)
MFCC Delta: (13, 194)
MFCC Delta-Delta: (13, 194)
Centres de bande de fréquences: (1, 194)


### Pression de seuil